In [75]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, LeaveOneOut, LeavePOut, RepeatedKFold, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler # robust_scale
from sklearn.svm import SVR

In [76]:
raw = pd.read_csv('../data/training_final_latcorr.csv')

In [77]:
raw.columns


Index(['field_1', 'Name', 'Longitude', 'Latitude', 'CO2 (mg C m¯² d¯¹)',
       'CH4 (mg C m-2 d-1)', 'CO2 (g/kWh)', 'CH4 (g/kWh)',
       'Area / Electricity', 'Area_km2', 'Age', 'Volume_km3',
       'Areakm2_div_Volkm3', 'org_c', 'temp_annual_avg',
       'temp_diff_summer_winter_lc', 'temp_spring_avg_lc', 'temp_spring_avg',
       'temp_summer_avg_lc', 'temp_summer_avg', 'temp_fall_avg_lc',
       'temp_fall_avg', 'temp_winter_avg_lc', 'temp_winter_avg',
       'NDVI_annual_avg', 'NDVI_spring_avg_lc', 'NDVI_spring_avg',
       'NDVI_summer_avg_lc', 'NDVI_summer_avg', 'NDVI_fall_avg_lc',
       'NDVI_fall_avg', 'NDVI_winter_avg_lc', 'NDVI_winter_avg',
       'npp_annual_avg', 'npp_spring_avg_lc', 'npp_spring_avg',
       'npp_summer_avg_lc', 'npp_summer_avg', 'npp_fall_avg_lc',
       'npp_fall_avg', 'npp_winter_avg_lc', 'npp_winter_avg', 'erosion',
       'precip'],
      dtype='object')

In [78]:
raw_cols_to_include = [
    #'field_1', 
    #'Name', 
    #'Longitude', 
    'Latitude', 
    #'CO2 (mg C m¯² d¯¹)',
    'CH4 (mg C m-2 d-1)', 
    #'CO2 (g/kWh)', 
    #'CH4 (g/kWh)',
    #'Area / Electricity', 
    'Area_km2', 
    'Age', 
    'Volume_km3',
    'Areakm2_div_Volkm3', 
    'org_c', 
    'temp_annual_avg', 
    'temp_diff_summer_winter_lc',
    #'temp_spring_avg_lc',
    #'temp_spring_avg', 
    #'temp_summer_avg_lc',
    #'temp_summer_avg',
    #'temp_fall_avg_lc', 
    #'temp_fall_avg', 
    #'temp_winter_avg_lc',
    #'temp_winter_avg', 
    'NDVI_annual_avg', 
    #'NDVI_spring_avg_lc',
    #'NDVI_spring_avg', 
    #'NDVI_summer_avg_lc', 
    #'NDVI_summer_avg',
    #'NDVI_fall_avg_lc', 
    #'NDVI_fall_avg', 
    #'NDVI_winter_avg_lc',
    #'NDVI_winter_avg', 
    'npp_annual_avg', 
    #'npp_spring_avg_lc',
    #'npp_spring_avg', 
    #'npp_summer_avg_lc', 
    #'npp_summer_avg',
    #'npp_fall_avg_lc', 
    #'npp_fall_avg', 
    #'npp_winter_avg_lc',
    #'npp_winter_avg',
    'erosion',
    'precip'
]

clean = raw[raw_cols_to_include].copy()

print("raw.shape   =", raw.shape)
print("clean.shape =", clean.shape)

raw.shape   = (154, 44)
clean.shape = (154, 13)


In [79]:
clean.rename(columns = {'CH4 (mg C m-2 d-1)':'ch4_emissions', 'CO2 (mg C m¯² d¯¹)':'co2_emissions'}, inplace = True) 

In [80]:
clean['log_ch4_emissions'] = np.log(clean['ch4_emissions'])

In [81]:
reduced = clean.dropna()
reduced.shape

(88, 14)

In [82]:
#target = 'CH4 (mg C m¯² d¯¹)'
target = 'log_ch4_emissions'

features = ['Age', 'org_c', 'temp_annual_avg', 'temp_diff_summer_winter_lc', 
            'NDVI_annual_avg', 'npp_annual_avg', 'erosion', 'precip']

X = reduced[features]
y = reduced[target]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [84]:
print("X_train.shape =", X_train.shape)
print("X_test.shape  =", X_test.shape)
print("y_train.shape =", y_train.shape)
print("y_test.shape  =", y_test.shape)

X_train.shape = (66, 8)
X_test.shape  = (22, 8)
y_train.shape = (66,)
y_test.shape  = (22,)


In [85]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rfr', RandomForestRegressor(random_state=0))
])

pipeline.fit(X_train, y_train)

print("Train score =", pipeline.score(X_train, y_train))
print("Test score  =", pipeline.score(X_test, y_test))

Train score = 0.9209885323437533
Test score  = 0.593547984584792


In [86]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rfr', RandomForestRegressor(random_state=0))
])

parameters = {
    'rfr__n_estimators': range(20, 200, 20),
}

my_cv = RepeatedKFold(n_splits=2, n_repeats=10, random_state=0)

best_ch4_model = GridSearchCV(pipeline, parameters, cv=my_cv, n_jobs=-1, scoring='r2') 

best_ch4_model.fit(X_train, y_train)

print("Best score: %0.3f" % best_ch4_model.best_score_)
best_parameters = best_ch4_model.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

best_ch4_model.best_estimator_.score(X_test, y_test)

Best score: 0.370
	rfr__n_estimators: 60


0.5863500395494796

## Use this model to predict CH4 emissions in USA dataset

In [87]:
present = pd.read_csv('data_predict/merged_PRESENT.csv')
best_case = pd.read_csv('data_predict/merged_2100ssp126.csv')
worst_case = pd.read_csv('data_predict/merged_2100ssp585.csv')

In [88]:
for column in present[features]:
    print(column, ':',  present[features][column].isna().sum())

Age : 0
org_c : 0
temp_annual_avg : 0
temp_diff_summer_winter_lc : 0
NDVI_annual_avg : 0
npp_annual_avg : 33
erosion : 0
precip : 0


In [89]:
present['npp_annual_avg'] = present['npp_annual_avg'].fillna(present['npp_annual_avg'].mean())
best_case['npp_annual_avg'] = best_case['npp_annual_avg'].fillna(best_case['npp_annual_avg'].mean())
worst_case['npp_annual_avg'] = worst_case['npp_annual_avg'].fillna(worst_case['npp_annual_avg'].mean())

In [90]:
present['ch4_emissions_logscale'] = best_ch4_model.predict(present[features])
best_case['ch4_emissions_logscale'] = best_ch4_model.predict(best_case[features])
worst_case['ch4_emissions_logscale'] = best_ch4_model.predict(worst_case[features])

In [91]:
present['ch4_emissions'] = 10**present['ch4_emissions_logscale']
best_case['ch4_emissions'] = 10**best_case['ch4_emissions_logscale']
worst_case['ch4_emissions'] = 10**worst_case['ch4_emissions_logscale']

In [92]:
present.to_csv('data_predict/merged_PRESENT_ch4predicted.csv')
best_case.to_csv('data_predict/merged_2100ssp126_ch4predicted.csv')
worst_case.to_csv('data_predict/merged_2100ssp585_ch4predicted.csv')

In [93]:
present['ch4_emissions'].median()

684.0259643086779

In [94]:
best_case['ch4_emissions'].median()

796.3405983030921

In [95]:
worst_case['ch4_emissions'].median()

1486.70566558934